In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-ADK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Compreensão do problema --- #
##########################################
def agente_inicial(topico2, data_de_hoje):

    inicial = Agent(
        name="agente_inicial",
        model="gemini-2.5-flash-preview-04-17",
        description="Agente de compreensão do problema e de estratégia inicial",
        tools=[google_search],
        instruction=f"""
        Você é um assistente de um advogado. A sua tarefa é compreender qual a
        situação apresentada pelo cliente do advogado, que será apresentada
        resumidamente no campo {topico2}.
        Com essas informações, crie uma estratégia com quais argumentos devem
        ser apresentados na petição inicial para que o cliente alcance o seu
        objetivo.
        Mantenha os termos técnicos da área do Direito, e crie no máximo seis
        argumentos.
        """
    )

    entrada_do_agente_inicial = f"Situação do cliente: {topico2}\nData de hoje: {data_de_hoje}"

    resultado_lancamentos = call_agent(inicial, entrada_do_agente_inicial)

    return resultado_lancamentos

    # Executa o agente

In [9]:
################################################
# --- Agente 2: Busca de subsídios na internet para embasar a estratégia criada --- #
################################################
def agente_planejador(topico2, lancamentos):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction=f"""
        Você é um assistente de um advogado. Considerando a situação do cliente e a estratégia criada,
        você deverá usar a ferramenta de busca do google (google_search) para
        procurar notícias, artigos e jurisprudências que sejam aplicáveis ao caso apresentado.
        A busca deverá ser realizada apenas com resultados dos seguintes sites:
        www.migalhas.com.br
        direitoreal.com.br/noticias
        www.jurisite.com.br/noticias_juridicas
        www.conjur.com.br
        www.jusbrasil.com.br
        Existindo informações contraditórias entre si nos resultados, você deverá dar preferência
        para as notícias mais recentes, considerando a {data_de_hoje}
        Apresente as cinco informações mais relevantes que você encontrar em cada um dos sites citados acima
        """,
        description="Agente que busca notícias e informações sobre o caso",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"situação do cliente:{topico2}\nestratégia criada: {resultado_lancamentos}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [17]:
######################################
# --- Agente 3: Revisor da estratégia e comparação com informações encontradas --- #
######################################
def agente_redator(topico2, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um advogado experiente e meticuluso, que está verificando o trabalho realizado no
            escritório de advocacia. Você deverá analisar se a estratégia criada para conseguir
            resolver a situação do cliente é compatível com as notícias e jurisprudências pesquisada.
            Você deverá sugerir melhorias na estratégia criada considerando a jurisprudência localizada
            """,
        description="Agente revisor e consolidador da estratégia"
    )
    entrada_do_agente_redator = f"situação do cliente: {topico2}\nestratégia criada: {resultado_lancamentos}\njurisprudência: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [18]:
##########################################
# --- Agente 4: Buscador de Jurisprudências --- #
##########################################
def agente_revisor(topico2, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um assistente de um advogado. Considerando a situação do cliente e o rascunho criado, verifique se há
            possibilidade para melhoras, indicando-as de forma resumida.
            Ainda, utilize a busca do google (google_search) dentro do site www.jusbrasil.com.br para encontrar
            10 jurisprudências recentes, preferencialmente dos últimos cinco anos, e que possuam pertinência e similariedade com o caso do cliente
            para serem citadas na petição inicial. A jurisprudência não poderá ser inventada, e deverá
            indicar o número completo do processo
            """,
        tools=[google_search],
        description="Agente buscador de juris."
    )
    entrada_do_agente_revisor = f"situação do cliente: {topico2}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [19]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("⚖️ Iniciando o Sistema de Auxílio de Estratégias Para PETIÇÕES INICIAIS ⚖️")

# --- Obter o Tópico do Usuário ---
topico2 = input("❓ Por favor, explique a situação do cliente para que possamos iniciar a análise do caso: ")

# Inserir lógica do sistema de agentes ################################################
if not topico2:
    print("Por favor, apresente o problema de seu cliente.")

else:
    print(f"""Obrigado! Aguarde enquanto realizamos a análise do caso e buscamos as
    mais recentes notícias e jurisprudências a respeito""")

    resultado_lancamentos = agente_inicial(topico2, data_de_hoje)
    print("\n--- Resultado do Agente 1 - Compreensão do problema e estratégias iniciais ---\n")
    display(to_markdown(resultado_lancamentos))
    print("---------------------------------------------")

    plano_de_post = agente_planejador(topico2, resultado_lancamentos)
    print("\n--- Resultado do Agente 2 - Entendimentos atuais e notícias sobre o tema ---\n")
    display(to_markdown(plano_de_post))
    print("---------------------------------------------")

    rascunho_de_post = agente_redator(topico2, plano_de_post)
    print("\n--- Resultado do Agente 3 - Estratégia a ser adotada ---\n")
    display(to_markdown(rascunho_de_post))
    print("---------------------------------------------")

    post_final = agente_revisor(topico2, rascunho_de_post)
    print("\n--- Resultado do Agente 4 - Jurisprudências ---\n")
    display(to_markdown(post_final))
    print("---------------------------------------------")



⚖️ Iniciando o Sistema de Auxílio de Estratégias Para PETIÇÕES INICIAIS ⚖️
❓ Por favor, explique a situação do cliente para que possamos iniciar a análise do caso: O cliente quer comprar um carro sem IPI
Obrigado! Aguarde enquanto realizamos a análise do caso e buscamos as
    mais recentes notícias e jurisprudências a respeito

--- Resultado do Agente 1 - Compreensão do problema e estratégias iniciais ---



> Compreendo a situação do cliente. Ele busca obter a isenção do Imposto sobre Produtos Industrializados (IPI) na compra de um veículo. Este benefício fiscal está previsto na Lei nº 8.989/1995 e é direcionado a categorias específicas, como pessoas com deficiência e taxistas. Considerando que a situação apresentada é concisa, a estratégia argumentativa na petição inicial deverá se basear nos requisitos legais para a concessão da isenção, na comprovação de que o cliente se enquadra em uma das categorias beneficiadas e nos princípios constitucionais que fundamentam tal direito.
> 
> Abaixo, apresento seis argumentos que podem ser utilizados na petição inicial, com base nas informações e na legislação pertinente:
> 
> 1.  **Do Direito à Isenção de IPI com Base na Lei nº 8.989/1995:** Argumentar que o cliente preenche os requisitos estabelecidos pela Lei nº 8.989/1995 para a concessão da isenção do IPI na aquisição de veículo novo. Detalhar qual inciso do Artigo 1º da referida lei se aplica ao caso concreto (por exemplo, se o cliente é pessoa com deficiência física, visual, mental severa ou profunda, autista, ou taxista). Juntar a documentação comprobatória necessária, como laudo médico circunstanciado ou comprovante da atividade de taxista.
> 
> 2.  **Da Natureza Extrafiscal do IPI e a Finalidade Social da Isenção:** Destacar o caráter extrafiscal do IPI, que permite ao Poder Executivo conceder isenções para atender a interesses sociais relevantes. Argumentar que a isenção em questão possui uma finalidade social clara, visando facilitar a mobilidade e a inclusão de pessoas com deficiência ou o exercício da profissão por taxistas, reduzindo desigualdades e promovendo a dignidade da pessoa humana.
> 
> 3.  **Da Violação aos Princípios Constitucionais da Isonomia e da Dignidade da Pessoa Humana:** Caso a isenção tenha sido negada administrativamente com base em interpretações restritivas da norma, argumentar que tal negativa viola os princípios constitucionais da isonomia e da dignidade da pessoa humana (Artigos 1º, III, e 5º, caput, da Constituição Federal). Sustentar que a lei de isenção deve ser interpretada de forma teleológica e sistemática, em consonância com o conjunto de normas que visam à proteção e inclusão das pessoas com deficiência e à valorização do trabalho.
> 
> 4.  **Da Desnecessidade de Anotação Restritiva na CNH para Pessoas com Deficiência (se aplicável ao caso):** Se a negativa se baseou na ausência de anotação restritiva na Carteira Nacional de Habilitação (CNH), argumentar, com base em jurisprudência consolidada do Superior Tribunal de Justiça (STJ), que a Lei nº 8.989/1995 não exige tal anotação para a concessão do benefício fiscal, sendo suficiente a comprovação da deficiência por meio de laudo médico.
> 
> 5.  **Da Aplicação da Jurisprudência Dominante do STJ:** Citar precedentes do Superior Tribunal de Justiça que reconhecem o direito à isenção de IPI em casos análogos ao do cliente, especialmente no que tange à interpretação extensiva do rol de deficiências contempladas e à finalidade social da norma.
> 
> 6.  **Do Cumprimento dos Requisitos Formais e Materiais:** Demonstrar que o cliente cumpriu todos os requisitos formais para a solicitação administrativa da isenção, bem como que preenche os requisitos materiais exigidos pela lei para a concessão do benefício. Juntar os protocolos dos requerimentos administrativos e a documentação comprobatória que ateste o direito pleiteado.


---------------------------------------------

--- Resultado do Agente 2 - Entendimentos atuais e notícias sobre o tema ---



> Compreendo a estratégia definida e vou realizar a busca de notícias, artigos e jurisprudências nos sites especificados, buscando informações relevantes para o caso do cliente que deseja comprar um carro com isenção de IPI.
> 
> 
> Com base nas buscas realizadas nos sites especificados, apresento as informações mais relevantes sobre a isenção de IPI na compra de veículos, aplicáveis ao caso do cliente:
> 
> **www.migalhas.com.br:**
> 
> 1.  Não foram encontrados resultados diretamente relevantes para a isenção de IPI para compra de veículos no site Migalhas.
> 
> **direitoreal.com.br/noticias:**
> 
> 1.  Não foram encontrados resultados diretamente relevantes para a isenção de IPI para compra de veículos no site Direitorio Real.
> 
> **www.jurisite.com.br/noticias\_juridicas:**
> 
> 1.  Não foram encontrados resultados diretamente relevantes para a isenção de IPI para compra de veículos no site Jurisite.
> 
> **www.conjur.com.br:**
> 
> 1.  Não foram encontrados resultados diretamente relevantes para a isenção de IPI para compra de veículos no site Conjur.
> 
> **www.jusbrasil.com.br:**
> 
> 1.  **Isenção para Pessoas com Deficiência:** Pessoas com deficiência física, visual, auditiva, mental severa ou profunda, ou com transtorno do espectro autista têm direito à isenção de IPI na compra de um carro a cada 3 anos. Motoristas profissionais (taxistas) podem solicitar a isenção a cada 2 anos.
> 2.  **Documentação Necessária:** Para pessoas com deficiência ou autismo, é necessário apresentar laudo médico.
> 3.  **Legislação:** A Instrução Normativa RFB nº 1.716/2017, a Instrução Normativa RFB nº 1.769/2017 e a Instrução Normativa RFB nº 2.022/2021 são relevantes para o processo de isenção.
> 4.  **Limite de Valor do Veículo:** A Lei nº 8.989/1995 estabelece regras para a isenção de IPI, incluindo um limite de valor para o veículo. Até 31 de dezembro de 2021, a aquisição com isenção aplicava-se a veículos novos com preço de venda ao consumidor não superior a R$ 140.000,00.
> 5.  **Requisitos para Taxistas:** Para motoristas profissionais (taxistas), é necessário constar na Carteira Nacional de Habilitação (CNH) que exerce atividade remunerada e não ter sofrido sanção ou condenação criminal que proíba receber benefícios fiscais.
> 
> É importante ressaltar que a legislação e os procedimentos para a isenção de IPI podem ser alterados, sendo crucial verificar as normas mais recentes e os requisitos específicos junto aos órgãos competentes antes de iniciar o processo.
> 


---------------------------------------------

--- Resultado do Agente 3 - Estratégia a ser adotada ---



> Excelente iniciativa em verificar a compatibilidade da estratégia com o material pesquisado. Como advogado experiente e meticuloso, procederei à análise, considerando estritamente as informações apresentadas na estratégia e na jurisprudência fornecida.
> 
> **Análise da Estratégia vs. Jurisprudência Fornecida:**
> 
> A estratégia elaborada está bem estruturada e abrange os pontos cruciais para pleitear a isenção do IPI, baseando-se corretamente na Lei nº 8.989/1995 e em princípios jurídicos relevantes. No entanto, ao confrontá-la com a jurisprudência (que, neste caso, consiste primariamente em informações descritivas do Jusbrasil, já que os outros sites não retornaram resultados diretos), identificamos alguns pontos de alinhamento e algumas áreas que poderiam ser aprimoradas ou melhor detalhadas com base no material disponível.
> 
> 1.  **Do Direito à Isenção (Lei 8.989/1995):** A estratégia baseia-se corretamente na lei e na necessidade de comprovar o enquadramento em uma das categorias. A jurisprudência fornecida corrobora isso, listando as categorias (PwD, Autistas, Taxistas) e a necessidade de documentação (laudo médico). **Ponto bem alinhado.**
> 2.  **Natureza Extrafiscal e Finalidade Social:** A estratégia aborda a finalidade social. A jurisprudência fornecida, ao listar os beneficiários, *implica* essa finalidade, mas não a discute explicitamente ou aprofunda o argumento extrafiscal. É um argumento jurídico válido, mas não está *diretamente suportado* pelo texto da jurisprudência fornecida.
> 3.  **Violação aos Princípios Constitucionais:** A estratégia invoca princípios como isonomia e dignidade. A jurisprudência fornecida *não menciona* princípios constitucionais. São argumentos jurídicos fortes e comuns em teses de direito tributário/constitucional, mas não derivam do material pesquisado neste caso específico.
> 4.  **Desnecessidade de Anotação Restritiva na CNH (PwD):** A estratégia *menciona* a jurisprudência do STJ sobre a desnecessidade para PwD. A jurisprudência fornecida *apenas menciona* a necessidade de anotação de atividade remunerada *para taxistas*. O material fornecido *não* corrobora o ponto específico sobre PwD e a jurisprudência do STJ a respeito. A estratégia está provavelmente correta sobre a posição do STJ para PwD, mas o material pesquisado *não a suporta*. Pelo contrário, o material *apresenta* um requisito de CNH, mas *apenas* para taxistas.
> 5.  **Aplicação da Jurisprudência Dominante do STJ:** A estratégia sugere citar precedentes do STJ. A jurisprudência fornecida *não apresenta* nenhum precedente específico do STJ, apenas descreve as regras baseadas em leis e INs. Portanto, este ponto da estratégia *não está suportado* pelo material pesquisado.
> 6.  **Cumprimento dos Requisitos Formais e Materiais:** A estratégia fala em demonstrar o cumprimento dos requisitos. A jurisprudência fornecida aponta alguns requisitos materiais (laudo médico, CNH para taxistas) e menciona INs que tratam dos procedimentos formais. **Ponto parcialmente alinhado/suportado.**
> 
> **Pontos Relevantes da Jurisprudência (Jusbrasil) que a Estratégia Não Aborda Explicitamente:**
> 
> *   **Instruções Normativas da RFB:** A jurisprudência menciona expressamente as INs RFB nº 1.716/2017, nº 1.769/2017 e nº 2.022/2021. Estas normas detalham os procedimentos administrativos, documentação e requisitos práticos para a solicitação da isenção. Sua menção e análise são cruciais.
> *   **Limite de Valor do Veículo:** A jurisprudência menciona um limite de R$ 140.000,00 para a isenção, destacando que essa regra era válida "Até 31 de dezembro de 2021". Esta é uma informação **extremamente relevante** que afeta diretamente o direito à isenção e a sua forma (total ou parcial, dependendo do valor atual do veículo e da legislação vigente após essa data). A estratégia não aborda o limite de valor.
> *   **Frequência da Isenção:** A jurisprudência menciona a frequência (3 anos para PwD, 2 anos para Taxistas). Embora implícito na lei, é um detalhe prático relevante.
> 
> **Sugestões de Melhoria na Estratégia (Considerando a Jurisprudência Fornecida):**
> 
> Com base estritamente no material pesquisado, sugiro as seguintes melhorias para integrar as informações encontradas e tornar a estratégia mais robusta e alinhada com o material disponível:
> 
> 1.  **Integração das Instruções Normativas:** No ponto 1 (ou em um novo ponto dedicado), inclua a menção às Instruções Normativas RFB nº 1.716/2017, nº 1.769/2017 e nº 2.022/2021 como base normativa complementar à Lei nº 8.989/1995. Argumente que o cliente atende aos requisitos *legais* e *normativos* para a isenção, detalhando como a documentação comprobatória (laudo médico, etc.) está em conformidade com o exigido por essas INs.
> 2.  **Abordagem do Limite de Valor do Veículo:** Crie um novo argumento ou sub-ponto dedicado à questão do limite de valor do veículo. Mencione a regra histórica (como a de R$ 140.000,00 até 31/12/2021, conforme a pesquisa) e, crucialmente, argumente com base na legislação *atual* aplicável ao caso do cliente, verificando qual é o limite de valor vigente e se o veículo pretendido se enquadra na isenção total ou parcial (se for o caso, após as alterações legislativas mais recentes). A omissão deste ponto é uma lacuna significativa, pois o valor do veículo é um requisito material fundamental.
> 3.  **Refinamento do Argumento da CNH:** Se o cliente for taxista, utilize a informação da jurisprudência fornecida (necessidade de anotação de atividade remunerada na CNH) como um requisito cumprido. Se o cliente for PwD, mantenha o argumento da desnecessidade de anotação restritiva para PwD com base na jurisprudência do STJ (mesmo que não esteja no material fornecido), mas reconheça a distinção e a relevância da CNH para a categoria de taxista, conforme a pesquisa indicou. Deixe claro que a argumentação sobre CNH é específica para a condição do cliente.
> 4.  **Ajuste sobre Jurisprudência STJ (no contexto do material):** Como o material fornecido não trouxe precedentes específicos do STJ, talvez o ponto 5 da estratégia precise ser reavaliado *se a petição for baseada APENAS nesse material*. No entanto, como advogado, você sabe que a jurisprudência do STJ é vital. Mantenha o ponto 5, mas esteja ciente de que o material de pesquisa *apresentado* não o suporta diretamente; você precisaria incorporar precedentes específicos reais.
> 5.  **Ênfase na Documentação e Procedimento Administrativo:** Reforce o ponto 6, destacando que a documentação juntada e o procedimento administrativo seguido estão em estrita conformidade com a Lei nº 8.989/1995 *e as Instruções Normativas RFB pertinentes* (citadas na pesquisa).
> 
> **Em resumo:** A estratégia é sólida em sua estrutura legal geral. As principais melhorias sugeridas, *baseadas no material de pesquisa fornecido*, são a **incorporação das Instruções Normativas** e, crucialmente, a **inclusão e análise do requisito do limite de valor do veículo**, verificando a regra atual aplicável ao caso do cliente. Além disso, a clareza sobre os requisitos de CNH, distinguindo PwD e Taxistas conforme a pesquisa, é importante.
> 
> Lembre-se sempre de verificar a legislação mais atualizada e a jurisprudência mais recente, pois as regras fiscais, especialmente limites de valor e procedimentos, podem ser alteradas. A pesquisa fornecida já sinaliza essa necessidade ("crucial verificar as normas mais recentes").
> 
> Espero que esta análise detalhada seja útil para refinar a petição inicial.


---------------------------------------------

--- Resultado do Agente 4 - Jurisprudências ---



> Para aprimorar ainda mais a sua petição inicial e garantir uma defesa robusta dos direitos do seu cliente, além das sugestões já apresentadas, é crucial enriquecer a peça com jurisprudências recentes e pertinentes. Abaixo, apresento uma busca direcionada no Jusbrasil para encontrar 10 jurisprudências que possam fortalecer seus argumentos, focando nos últimos cinco anos e na similaridade com o caso de isenção de IPI para compra de veículo.
> 
> 
> Com base nas suas instruções e nos resultados da pesquisa no Jusbrasil, apresento algumas sugestões para aprimorar seu rascunho, juntamente com jurisprudências relevantes dos últimos cinco anos (aproximadamente), que podem ser citadas na petição inicial:
> 
> **Sugestões de Melhorias no Rascunho:**
> 
> 1.  **Atualização Normativa:** Dada a dinamicidade da legislação tributária, reforce a necessidade de verificar as normas mais recentes, especialmente as Instruções Normativas da Receita Federal, que detalham os procedimentos e requisitos para a isenção de IPI, ICMS e IPVA.
> 2.  **Limite de Valor do Veículo:** Inclua uma seção específica sobre o limite de valor do veículo, mencionando que este limite é frequentemente atualizado e que a legislação vigente no momento da aquisição é determinante para a concessão da isenção.
> 3.  **Requisitos Específicos:** Detalhe os requisitos específicos para cada tipo de deficiência ou condição (PCD, taxistas, etc.), com base na legislação aplicável e nas instruções normativas da Receita Federal.
> 4.  **Jurisprudência do STJ:** Inclua jurisprudência atualizada do STJ sobre a isenção de IPI para PCDs, mencionando os requisitos e condições estabelecidos pelo tribunal.
> 5.  **Visão Monocular:** Se aplicável, mencione a isenção de IPVA para pessoas com visão monocular, citando jurisprudência que reconhece esse direito.
> 6.  **Cumulação de Isenções:** Explique se é possível cumular as isenções de IPI, ICMS e IPVA, e quais são os requisitos para cada uma delas.
> 7.  **Ação Declaratória:** Considere a possibilidade de ingressar com uma ação declaratória para garantir o direito à isenção, especialmente se houver negativa por parte da administração pública.
> 
> **Jurisprudências Relevantes (Jusbrasil):**
> 
> Abaixo estão 10 exemplos de jurisprudências encontradas no Jusbrasil que podem ser relevantes para o seu caso. É importante verificar a aplicabilidade de cada uma delas ao caso concreto do seu cliente:
> 
> 1.  **Processo nº (não especificado):** Ação Declaratória de Inexistência de Relação Jurídico-Tributária c/c Repetição de Indébito. Menciona a prioridade de tramitação para portadores de deficiência e a isenção de IPI.
> 2.  **Processo nº (não especificado):** Mandado de Segurança para isenção de ICMS e IPVA para PCD, com pedido de tutela de urgência.
> 3.  **Processo nº 0027616-67.2020.8.16.0182 (TJPR):** Isenção de IPI para PCD. Considera o laudo médico emitido para concessão da isenção do IPI como suficiente para comprovar a deficiência.
> 4.  **Processo nº (não especificado):** Ação anulatória tributária sobre ICMS, mencionando a Súmula 546 do STF sobre restituição de tributo pago indevidamente.
> 5.  **Processo nº (não especificado):** Caso sobre isenção de IPVA e ICMS para PCD, com pedido de prioridade na tramitação.
> 6.  **Processo nº (não especificado):** Discussão sobre o limite de valor para a isenção de ICMS, com base na Lei 6.466/19.
> 7.  **Processo nº (não especificado):** Caso sobre veículo adquirido por taxista com isenção de IPI e ICMS, mas não entregue.
> 8.  **Processo nº (não especificado):** Ação buscando a restituição de valores descontados indevidamente nos últimos cinco anos.
> 9.  **Processo nº (não especificado):** Decisão do TRF4 garantindo isenção de IPI na compra de carro adaptado para mulher com limitação no joelho.
> 10. **Processo nº (não especificado):** Caso sobre a não homologação da compensação de débitos, resultando na constituição do crédito.
> 
> **Observações Importantes:**
> 
> *   **Legislação Estadual:** Verifique a legislação do estado do seu cliente para confirmar as regras sobre isenção de ICMS e IPVA, que podem variar de um estado para outro.
> *   **Requisitos Específicos:** Certifique-se de que o seu cliente cumpre todos os requisitos específicos para a isenção, como laudos médicos, comprovantes de residência, etc.
> *   **Valor do Veículo:** Verifique o limite de valor do veículo estabelecido pela legislação vigente no momento da aquisição.
> *   **Atualização:** Mantenha-se atualizado sobre as decisões dos tribunais superiores (STJ e STF) sobre a matéria, que podem influenciar o resultado do caso.
> 
> Lembre-se que esta é apenas uma sugestão inicial e que cada caso é único. É fundamental analisar cuidadosamente a situação do seu cliente e a legislação aplicável para determinar a melhor estratégia a ser adotada.
> 


---------------------------------------------
